In [ ]:
# Crea, entrena y guarda un modelo de clasificación de imágenes vulcanológicas.
# Puede tardar bastante tiempo (una hora o más) y requiere de la gran mayoría de la potencia de la computadora mientras se ejecuta.
# La estructura está basada en la arquitectura que Francois Chollet recomienda para este tipo de sets de datos "limitados"

In [1]:
# importa todas las librerias necesarias
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
# Define la normalizacion de las dimensiones de las imágenes
img_width = 255
img_height = 255

train_data_dir = 'images_crop/train/'
validation_data_dir = 'images_crop/validation/'

# es el número de imágenes en train y validation
nb_train_samples = 227 + 39
nb_validation_samples = 9 + 15

# convierte el formato

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [3]:
#  crear el modelo que tendrá de output un "feature map"(height, width, features)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))



#






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
#  Define el tipo de compilación del modelo
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
# Agrega transformaciones a las imagenes de entrenamiento para agrandar el set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [6]:
# Las "epochs" definen la cantidad de rounds de entrenamiento. Incrementarlo puede ocasionar problemas si no se observan cuidadosamente las métricas
epochs = 200
batch_size = 16

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 324 images belonging to 2 classes.


In [7]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 34 images belonging to 2 classes.


In [8]:
# Esta es la parte donde se tardará el 99% del tiempo en ejecutarse, e irá imprimendo las métricas de entrenamiento
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)




Epoch 1/200





16/16 [==============================] - 15s 956ms/step - loss: 0.6709 - acc: 0.7891 - val_loss: 1.2337 - val_acc: 0.5625
Epoch 2/200
16/16 [==============================] - 12s 734ms/step - loss: 0.4944 - acc: 0.8472 - val_loss: 0.6838 - val_acc: 0.6250
Epoch 3/200
16/16 [==============================] - 12s 735ms/step - loss: 0.5188 - acc: 0.8447 - val_loss: 0.1900 - val_acc: 1.0000
Epoch 4/200
16/16 [==============================] - 11s 663ms/step - loss: 0.4561 - acc: 0.8205 - val_loss: 0.2743 - val_acc: 0.8750
Epoch 5/200
16/16 [==============================] - 11s 666ms/step - loss: 0.3869 - acc: 0.8516 - val_loss: 0.9439 - val_acc: 0.3125
Epoch 6/200
16/16 [==============================] - 10s 630ms/step - loss: 0.3917 - acc: 0.8330 - val_loss: 0.0115 - val_acc: 1.0000
Epoch 7/200
16/16 [==============================] - 11s 662ms/step - loss: 0.4259 - acc: 0.8315 - val_loss: 0.9875 - val_acc: 0.5000
Epoch 8/200
16/16 [==============================] - 1

In [9]:
# Guarda el modelo en un archivo nuevo
# si no se cambia el nombre, se sobreescribe el modelo anterior
# si sí, se crea uno nuevo con ese nombre
model.save('volcano_analysis1.0.h5')

In [57]:
# Genera datos de una prueba casi idéntica a clasifica.py
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'test_images/',
        target_size=(255, 255),
        color_mode="rgb",
        shuffle = False,
        class_mode='binary',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)

Found 10 images belonging to 1 classes.


In [58]:
# Imprime las clasificaciones en orden
print(predict)

[[1.0000000e+00]
 [1.0000000e+00]
 [1.0000000e+00]
 [1.0000000e+00]
 [1.0000000e+00]
 [5.4585020e-04]
 [2.4906127e-08]
 [8.7574035e-02]
 [0.0000000e+00]
 [3.4938570e-02]]
